In [16]:
import requests, json, math, re

In [29]:
download()

{b'BNB': {b'BTC': 6.281709901962928, b'ETH': 3.723967916786431},
 b'BTC': {b'BNB': -6.281705841954686, b'ETH': -2.5580796611925813},
 b'ETH': {b'BNB': -3.723763855967438, b'BTC': 2.5580968893409852}}

In [28]:
binance = {
 'BTC_BTC': '1',
 'BTC_BNB': '534.7',
 'BTC_ETH': '12.911',
 'BNB_BTC': '0.0018702',
 'BNB_BNB': '1',
 'BNB_ETH': '0.024138',
 'ETH_BTC': '0.077452',
 'ETH_BNB': '41.42',
 'ETH_ETH': '1',
}

In [43]:
def getRates():
    url = 'http://fx.priceonomics.com/v1/rates/?q=1'
    response = requests.get(url)
    rates = response.json()    
    return rates

def download():
    graph = {}
    rates = getRates()
    pattern = re.compile("([A-Z]{3})_([A-Z]{3})")
    for key in rates:
        matches = pattern.match(key)
        conversion_rate = -math.log(float(rates[key]))
        from_rate = matches.group(1).encode('ascii','ignore')
        to_rate = matches.group(2).encode('ascii','ignore')
        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)
    return graph

In [44]:

def initialize(graph, source):
    destination = {}
    predecessor = {}
    for node in graph:
        destination[node] = float('Inf')
        predecessor[node] = None
    destination[source] = 0
    return destination, predecessor


def relax(node, neighbour, graph, destination, predecessor):
    if destination[neighbour] > destination[node] + graph[node][neighbour]:
        destination[neighbour] = destination[node] + graph[node][neighbour]
        predecessor[neighbour] = node

def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        try:
            next_node = p[next_node]
            if next_node not in arbitrageLoop:
                arbitrageLoop.append(next_node)
            else:
                arbitrageLoop.append(next_node)
                arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
                return arbitrageLoop
        except:
            return arbitrageLoop

def bellman_ford(graph, source):
    destination, predecessor = initialize(graph, source)
    for i in range(len(graph)-1):
        for u in graph:
            for v in graph[u]:
                relax(u, v, graph, destination, predecessor)

    for u in graph:
        for v in graph[u]:
            return(retrace_negative_loop(predecessor, source))

    return None

In [45]:

paths = []

graph = download()

for key in graph:
    path = bellman_ford(graph, key)
    if path not in paths and not None:
        paths.append(path)

    
for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        X = 100
        print("X =  ",X," in ",path[0])

        for i,value in enumerate(path):
            if i+1 < len(path):
                start = path[i]
                end = path[i+1]
                rate = math.exp(-graph[start][end])
                X *= rate
                print("(start) ", start, " to (end)",end," at (rate)",rate," = (",path[0],")",X)                
print("\n")

X =   100  in  b'USD'
(start)  b'USD'  to (end) b'EUR'  at (rate) 0.8005527  = ( b'USD' ) 80.05527000000001
(start)  b'EUR'  to (end) b'USD'  at (rate) 1.4609771  = ( b'USD' ) 116.958916204317
X =   100  in  b'EUR'
(start)  b'EUR'  to (end) b'USD'  at (rate) 1.4609771  = ( b'EUR' ) 146.09771
(start)  b'USD'  to (end) b'EUR'  at (rate) 0.8005527  = ( b'EUR' ) 116.958916204317


